In [30]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import DictVectorizer

titanic=pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [31]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


In [32]:
x=titanic[['pclass','age','sex']]
y=titanic['survived']

In [33]:
'''
通过观察数据可以知道age数据列中，有空数据，我们使用数据在的平均值或中位数来替代可以对模型偏离造成最小影响，age我们需要转换为数值特征，我们可以转换为0/1,

'''
x['age'].fillna(x['age'].mean(),inplace=True)
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


C:\Users\rayshea\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [34]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=33)
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
print(vec.feature_names_)

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


In [35]:
X_test = vec.transform(X_test.to_dict(orient='record'))
X_test

array([[ 18.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ],
       [ 48.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [ 31.19418104,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ],
       ..., 
       [ 20.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   1.        ],
       [ 54.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ],
       [ 31.19418104,   0.        ,   0.        ,   1.        ,
          1.        ,   0.        ]])

In [36]:
#导入决策树模型并对测试特征数据进行预测
from sklearn.tree import DecisionTreeClassifier
#使用默认配置初始化决策树分类器
dtc = DecisionTreeClassifier()
#训练数据进行模型学习
dtc.fit(X_train,y_train)
#决策树模型对特征数据进行预测
y_predict=dtc.predict(X_test)

In [37]:
#模型评估
from sklearn.metrics import classification_report
#输出预测准确性
print(dtc.score(X_test,y_test))
#输出更加详细的分类性能
print(classification_report(y_predict,y_test,target_names=['died','survived']))

0.781155015198
             precision    recall  f1-score   support

       died       0.91      0.78      0.84       236
   survived       0.58      0.80      0.67        93

avg / total       0.81      0.78      0.79       329



In [38]:
#使用随机森林分类器进行集成模型的训练以及预测分析
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc_y_pred=rfc.predict(X_test)
#输出随机森林分类器在测试集上的分类准确性性
print(rfc.score(X_test,y_test))
print(classification_report(rfc_y_pred,y_test))

0.77811550152
             precision    recall  f1-score   support

          0       0.90      0.78      0.83       233
          1       0.59      0.78      0.67        96

avg / total       0.81      0.78      0.79       329



In [39]:
#使用梯度提升决策树进行集成O型的训练及预测分析
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
gbc_y_pred=gbc.predict(X_test)
#输出梯度提升树在测试集上的分类准确性
print(gbc.score(X_test,y_test))
print(classification_report(gbc_y_pred,y_test))

0.790273556231
             precision    recall  f1-score   support

          0       0.92      0.78      0.84       239
          1       0.58      0.82      0.68        90

avg / total       0.83      0.79      0.80       329



In [40]:
#方案二：分离数据特征与预测目标
X=titanic.drop(['row.names','name','survived'],axis=1)
#填充age缺失值，使用平均数或中位数
X['age'].fillna(X['age'].mean(),inplace=True)
#查看数据特征
X.info()
#填充缺失值
X.fillna('UNKNOWN',inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 8 columns):
pclass       1313 non-null object
age          1313 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), object(7)
memory usage: 82.1+ KB
